## 데코레이터 연습 파일

In [ ]:
## 가장 간단한 custom 데코레이터 예제

# 함수를 인자로 받는 함수
def deco(f):
    print("deco!")
    return f    # 원래 함수를 그대로 반환

In [6]:
@deco
def double(x) :
    return x * 2

deco!


In [9]:
print(double(10))

20


In [ ]:
## 데코레이터를 사용하지 않고 직접 적용할 수도 있음

def double(x) :
    return x * 2

double = deco(double)

deco!


In [8]:
print(double(10))

20


In [ ]:
## 함수가 아니라 다른 걸 반환해도 됨

def deco(f):
    print("deco!")
    return 10

@deco
def double(x) :
    return x * 2


deco!


In [5]:
double

10

In [ ]:
## 함수를 변형한 새 함수를 만들 수 있음

def deco(fn):
    def deco_hello():
        print("*" * 20)
        fn()
        print("*" * 20)
    return deco_hello

@deco
def hello():
    print("hello")

In [22]:
hello()

********************
hello
********************


In [ ]:
## 클래스로 데코레이터 만들기

class Deco():
    def __init__(self, n):
        print("deco init")
        self.n = n
    
    def __call__(self, f):
        print("deco call")
        return f
        

In [ ]:
@Deco(3)
def double(x) :
    return x * 2

deco init
deco call


In [21]:
double(10)

20

In [22]:
class Deco():
    def __init__(self, n):
        print("deco init")
        self.n = n
    
    def __call__(self, f):
        print("deco call")
        def newf(*args, **kwargs):
            print("deco newf")
            return f(*args, **kwargs) * self.n  
        return newf
        

In [23]:
@Deco(3)
def double(x) :
    return x * 2

deco init
deco call


In [29]:
ans = double(5)
print(ans)

deco newf
30


In [ ]:
## Timer 데코레이터 예제

import time

def timer(func):                            # 기존 함수를 인수로 받는다.
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)      # 기존 함수를 수행한다.
        end = time.time()
        print(f"{func.__name__} 실행 시간: {end - start:.4f}초")    # 기존 함수의 수행시간을 출력한다.
        return result                       # 기존 함수의 수행 결과를 리턴한다.
    return wrapper

@timer
def slow_function():
    print("함수가 실행됩니다.")
    time.sleep(1)

slow_function()


함수가 실행됩니다.
slow_function 실행 시간: 1.0015초


In [ ]:
## Login 여부 확인 데코레이터 예제

def require_login(func):
    def wrapper(user, *args, **kwargs):
        if not user.get("is_authenticated"):
            print("로그인이 필요합니다.")
            return None
        return func(user, *args, **kwargs)
    return wrapper

@require_login
def view_profile(user):
    print(f"{user['name']}의 프로필입니다.")

user1 = {"name": "Alice", "is_authenticated": True}
user2 = {"name": "Bob", "is_authenticated": False}

view_profile(user1)  # 출력: Alice의 프로필입니다.
view_profile(user2)  # 출력: 로그인이 필요합니다.


Alice의 프로필입니다.
로그인이 필요합니다.


In [ ]:
## Repeat 데코레이터 예제

def repeat(n):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(n):
                func(*args, **kwargs)
        return wrapper
    return decorator

@repeat(3)
def greet():
    print("Hi!")

greet()

Hi!
Hi!
Hi!


In [ ]:
## staticmethod 데코레이터 예제

class Math:
    @staticmethod
    def add(x, y):
        return x + y

print(Math.add(3, 5))

8


In [ ]:
## classmethod 데코레이터 예제

class Person:
    def __init__(self, name):
        self.name = name

    @classmethod
    def from_string(cls, s):
        name = s.strip().title()
        return cls(name)

p = Person.from_string(" alice ")
print(p.name)

Alice


In [ ]:
## property 데코레이터 예제 - getter, setter

class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def diameter(self):
        return self._radius * 2

    @diameter.setter
    def diameter(self, d):
        self._radius = d / 2

c = Circle(5)
print(c.diameter)
c.diameter = 20
print(c._radius)

10
10.0


In [ ]:
## property 데코레이터 예제 - deleter

class User:
    def __init__(self, username, password):
        self._username = username
        self._password = password

    @property
    def password(self):
        raise AttributeError("Password is write-only")

    @password.setter
    def password(self, new_password):
        if len(new_password) < 6:
            raise ValueError("Password too short")
        self._password = new_password

    @password.deleter
    def password(self):
        print("Deleting password...")
        del self._password

u = User("alice", "secret123")
del u.password   # → "Deleting password..."
print(u._password)  # AttributeError: '_password' is gone


Deleting password...


AttributeError: 'User' object has no attribute '_password'

In [ ]:
## contextmanager 데코레이터 예제

from contextlib import contextmanager
import os

@contextmanager
def open_file(path, mode):
    # 폴더가 있는 지 검사해서 없으면 만들어주는 context manager
    dir_name = os.path.dirname(path)
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    print(1)
    
    f = open(path, mode)
    try:
        yield f
    finally:    
        f.close()
        print(3)

with open_file("../example.txt", "w") as f:
    f.write("Hello, context manager!")
    print(2)

1
2
3


In [ ]:
## contextmanager 데코레이터 예제 2

@contextmanager
def ignoring(*exceptions):
    try:
        yield
    except exceptions as e:
        print(f"Ignored exception: {e}")
        pass

arr = [1,2]

# 이 안에서는 IndexError를 무시한다.
with ignoring(IndexError):
    print(arr[3])

Ignored exception: list index out of range


In [ ]:
## wraps 데코레이터 예제

from functools import wraps
def log1(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

def log2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

@log1
def greet1():
    """Greet the user"""
    print("Hello!")

@log2
def greet2():
    """Greet the user"""
    print("Hello!")

greet1()
print("----")
greet2()
print("====")
print(greet1)
print("----")
print(greet2)
print("====")
print(greet2.__name__)
print(greet2.__doc__)

Calling greet1
Hello!
----
Calling greet2
Hello!
====
<function log1.<locals>.wrapper at 0x000001E2410F04A0>
----
<function greet2 at 0x000001E2410D3420>
====
greet2
Greet the user


In [ ]:
## LRU Cache 데코레이터 예제

from time import sleep
from functools import lru_cache

@lru_cache()
def think(idea):
    print(f"Thinking about {idea}...")
    sleep(2)
    return "NO"

print(think("Python"))
print(think("Java"))    # 새로운 아이디어는 다시 생각해야 함
print(think("Python"))  # 캐시된 결과를 사용하여 빠르게 반환

Thinking about Python...
NO
Thinking about Java...
NO
NO


In [ ]:
## LRU Cache와 동일기능 함수 예제

def think2(idea, cache=dict()):
    if idea in cache:
        return cache[idea]
    print(f"Thinking about {idea}...")
    sleep(2)
    result = "NO"
    cache[idea] = result
    return result

print(think2("Python"))
print(think2("Java"))    # 새로운 아이디어는 다시 생각해야 함
print(think2("Python"))  # 캐시된 결과를 사용하여 빠르게 반환

Thinking about Python...
NO
Thinking about Java...
NO
NO
